In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

x = 10

## Summary Statistics function

In [2]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

def summary_statistics_g(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'saves': 'sum',
        'shots': 'sum',
        'start': 'sum',
        'shutout': 'sum',
        'fantasyPoints': 'sum'
    }).reset_index()
    
    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)
    
    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

# Pull in the data files from scrape

In [3]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=5)).strftime('%Y-%m-%d')}.xlsx"
# file_name = f"data/manual-adjustments.xlsx"
rankings = pd.read_excel(file_name, index_col=False)

# fileName = f"data/summary_stats_last{x}-{today.strftime('%Y-%m-%d')}.csv"
# last_10 = pd.read_csv(fileName, index_col=False)

In [4]:
ordercolumns = rankings.columns.to_list()
ordercolumns

['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-12-11',
 'rank_2023-12-04',
 'rank_2023-11-27',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01']

## Code to re-tag rankings for save

In [49]:
# file_name = f"data/manual-adjustments.xlsx"
# manual = pd.read_excel(file_name)
# for index, row in rankings.iterrows():
#     try:
#         rankings.at[index, 'rank_2023-11-27'] = player_bios_adjusted.loc[player_bios_adjusted['espnId'] == row['espnId'], 'rank'].iloc[0]
#     except:
#         rankings.at[index, 'rank_2023-11-27'] = 800

In [50]:
# fileName = f"data/rankings_worksheet_2023-11-27.xlsx"
# rankings.to_excel(fileName, index=False)

## New column order

In [5]:
column_order = ['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-12-18',            
 'rank_2023-12-11',
 'rank_2023-12-04',                
 'rank_2023-11-27',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01'
]

In [6]:
player_bios

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId
0,8479318,A. Matthews,TOR,C,35008,27,95.9,9.86,3.55,0,54,191.70,Auston Matthews,1997-09-17,L,75,215,99.907626,C,4024123
1,8481559,J. Hughes,NJD,C,28599,23,76.1,9.58,3.31,0,54,178.74,Jack Hughes,2001-05-14,L,71,175,99.775460,C,4565222
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,50,171.00,Nikita Kucherov,1993-06-17,L,71,182,99.877782,RW,2563060
3,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,54,166.32,Connor McDavid,1997-01-13,L,73,194,99.883457,C,3895074
4,8475660,C. Talbot,LAK,G,71457,20,80.2,4.04,4.01,0,40,162.64,Cam Talbot,1987-07-05,L,76,200,86.274426,G,5734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,8480992,M. Chrona,SJS,G,1830,1,-5.4,-10.62,-5.40,0,0,-9.37,Magnus Chrona,2000-08-28,L,76,194,0.022773,G,0
853,8477293,A. Raanta,CAR,G,44928,14,-9.2,-0.74,-0.66,0,21,-15.96,Antti Raanta,1989-05-12,L,72,195,5.982979,G,3037703
854,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,8,-16.67,Jack Campbell,1992-01-09,L,75,200,2.226018,G,5473
855,8477480,E. Comrie,BUF,G,20642,7,-13.2,-2.30,-1.89,0,9,-20.75,Eric Comrie,1995-07-06,L,73,183,0.240515,G,3042057


# Adjusted Fantasy Remain, done with Last 10

In [7]:
for index, row in player_bios.iterrows():
    if row['position'] != 'G':
        player_log = all_df.loc[all_df['playerId'] == row['playerId']]
        player_log.sort_values('gameDate', inplace=True, ascending=False)
        player_log = player_log.head(10)
        last_10_sum = summary_statistics(player_log)
        try:
            last_10_FPPG = last_10_sum['FPPG'].iloc[0]
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'last10FP'] = last_10_sum['fantasyPoints'].iloc[0]
            player_bios.at[index, 'adjFantasyRemain'] = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
        except:
            player_bios.at[index, 'last10FPPG'] = 0
            player_bios.at[index, 'last10FP'] = 0
            player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
    else:
        player_log = allG_df.loc[allG_df['playerId'] == row['playerId']]
        player_log.sort_values('gameDate', inplace=True, ascending=False)
        player_log = player_log.head(10)
        last_10_sum = summary_statistics_g(player_log)
        try:
            last_10_FPPG = last_10_sum['FPP60'].iloc[0]
            player_bios.at[index, 'last10FP'] = last_10_sum['fantasyPoints'].iloc[0]
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'adjFantasyRemain'] = (((row['FPP60'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
        except:
            player_bios.at[index, 'last10FPPG'] = 0
            player_bios.at[index, 'last10FP'] = 0
            player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']

player_bios.sort_values('adjFantasyRemain')

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,last10FP,adjFantasyRemain
856,8482821,A. Soderblom,CHI,G,45909,14,-23.6,-1.85,-1.69,0,...,1999-08-19,L,75,180,0.721605,G,4894729,-1.90,-17.4,-42.933333
855,8477480,E. Comrie,BUF,G,20642,7,-13.2,-2.30,-1.89,0,...,1995-07-06,L,73,183,0.240515,G,3042057,-2.30,-13.2,-20.700000
853,8477293,A. Raanta,CAR,G,44928,14,-9.2,-0.74,-0.66,0,...,1989-05-12,L,72,195,5.982979,G,3037703,-1.43,-13.2,-20.370000
854,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,1992-01-09,L,75,200,2.226018,G,5473,-1.93,-8.6,-15.440000
849,8475831,P. Grubauer,SEA,G,55322,17,-2.6,-0.17,-0.15,2,...,1991-11-25,L,73,188,5.093937,G,5657,-0.88,-7.4,-8.946667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,...,1993-06-17,L,71,182,99.877782,RW,2563060,3.10,31.0,165.666667
17,8476945,C. Hellebuyck,WPG,G,79316,22,75.4,3.42,3.43,0,...,1993-05-19,L,76,207,99.454882,G,3020225,5.77,57.6,168.133333
1,8481559,J. Hughes,NJD,C,28599,23,76.1,9.58,3.31,0,...,2001-05-14,L,71,175,99.775460,C,4565222,3.47,34.7,181.620000
3,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,...,1997-01-13,L,73,194,99.883457,C,3895074,4.14,41.4,185.400000


In [9]:
player_bios.sort_values('adjFantasyRemain', ascending=False).head(50)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,last10FP,adjFantasyRemain
0,8479318,A. Matthews,TOR,C,35008,27,95.9,9.86,3.55,0,...,1997-09-17,L,75,215,99.907626,C,4024123,3.83,38.3,196.740000
3,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,...,1997-01-13,L,73,194,99.883457,C,3895074,4.14,41.4,185.400000
1,8481559,J. Hughes,NJD,C,28599,23,76.1,9.58,3.31,0,...,2001-05-14,L,71,175,99.775460,C,4565222,3.47,34.7,181.620000
17,8476945,C. Hellebuyck,WPG,G,79316,22,75.4,3.42,3.43,0,...,1993-05-19,L,76,207,99.454882,G,3020225,5.77,57.6,168.133333
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,...,1993-06-17,L,71,182,99.877782,RW,2563060,3.10,31.0,165.666667
6,8477492,N. MacKinnon,COL,C,40820,30,91.9,8.10,3.06,0,...,1995-09-01,R,72,200,99.903354,C,3041969,3.36,33.6,164.320000
10,8480865,N. Dobson,NYI,D,45996,30,89.3,6.99,2.98,0,...,2000-01-07,R,76,200,96.766721,D,4352732,3.25,32.5,159.640000
11,8480803,E. Bouchard,EDM,D,37541,28,78.7,7.55,2.81,0,...,1999-10-20,R,75,192,96.843197,D,4352722,3.21,32.1,158.940000
15,8475786,Z. Hyman,EDM,L,31514,27,72.4,8.27,2.68,0,...,1992-06-09,R,73,206,94.087550,RW,5509,3.39,33.9,157.500000
4,8475660,C. Talbot,LAK,G,71457,20,80.2,4.04,4.01,0,...,1987-07-05,L,76,200,86.274426,G,5734,3.51,35.0,154.533333


# Get adjustment to Fantasy Remain

In [10]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [11]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)
opp_defense

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,200127,70,17,256,146,245,278.7,29,24,9.61,8,Anaheim Ducks
1,ARI,209196,48,18,274,220,257,269.9,30,13,9.00,16,Arizona Coyotes
2,BOS,200822,34,7,240,222,280,240.2,29,11,8.28,29,Boston Bruins
3,BUF,223840,51,18,234,187,291,282.1,32,12,8.82,19,Buffalo Sabres
4,CAR,208324,41,14,182,187,305,264.9,30,11,8.83,18,Carolina Hurricanes
5,CBJ,224081,61,15,311,151,331,309.2,32,12,9.66,6,Columbus Blue Jackets
6,CGY,218491,56,15,249,191,325,299.0,31,11,9.65,7,Calgary Flames
7,CHI,201913,67,12,250,169,252,269.4,29,17,9.29,14,Chicago Blackhawks
8,COL,208322,47,9,220,212,300,264.2,30,10,8.81,20,Colorado Avalanche
9,DAL,202518,49,15,255,141,293,271.1,29,9,9.35,12,Dallas Stars


In [12]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,104495,71,854,97.6,29,636,684,1,113,133,3.37,4,Anaheim Ducks
1,ARI,107706,88,798,32.0,30,572,633,1,124,148,1.07,24,Arizona Coyotes
2,BOS,104015,88,900,30.4,29,652,718,0,129,151,1.05,25,Boston Bruins
3,BUF,114882,83,957,91.8,32,729,799,2,129,141,2.87,7,Buffalo Sabres
4,CAR,108407,95,1018,57.6,30,772,840,1,123,145,1.92,16,Carolina Hurricanes
5,CBJ,114800,92,941,81.8,32,698,776,2,130,144,2.56,11,Columbus Blue Jackets
6,CGY,112385,87,968,84.2,31,748,818,1,111,122,2.72,10,Calgary Flames
7,CHI,103743,64,787,104.6,29,600,655,2,94,103,3.61,2,Chicago Blackhawks
8,COL,107734,98,957,33.8,30,683,754,3,156,178,1.13,23,Colorado Avalanche
9,DAL,104526,96,850,16.8,29,626,696,2,101,122,0.58,30,Dallas Stars


In [15]:
remainSked = homeOnlySked.loc[homeOnlySked['gameDT'] >= str((datetime.utcnow() + timedelta(days=0.5)))]
remainSked.sort_values('gameDate', inplace=True)
remainSked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
301,2023020482,2023-12-18,Monday 08:00 PM,SEA,DAL,2023-12-19 01:00:00,0
916,2023020483,2023-12-18,Monday 09:30 PM,FLA,CGY,2023-12-19 02:30:00,0
874,2023020480,2023-12-18,Monday 07:00 PM,MIN,PIT,2023-12-19 00:00:00,0
261,2023020481,2023-12-18,Monday 07:30 PM,MTL,WPG,2023-12-19 00:30:00,0
549,2023020479,2023-12-18,Monday 07:00 PM,ANA,DET,2023-12-19 00:00:00,0
...,...,...,...,...,...,...,...
286,2023021308,2024-04-18,Thursday 08:00 PM,VAN,WPG,2024-04-19 00:00:00,0
942,2023021309,2024-04-18,Thursday 09:00 PM,SJS,CGY,2024-04-19 01:00:00,0
40,2023021311,2024-04-18,Thursday 10:00 PM,ANA,VGK,2024-04-19 02:00:00,0
1024,2023021307,2024-04-18,Thursday 07:00 PM,SEA,MIN,2024-04-18 23:00:00,0


In [16]:
team_names_FPAPG = team_names.copy()
team_names_FPAPG['G_FPAPG'] = 0
team_names_FPAPG['F_FPAPG'] = 0
team_names_FPAPG['D_FPAPG'] = 0

for index, row in remainSked.iterrows():
    index_of_away = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['awayTeam']].index[0]
    index_of_home = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['homeTeam']].index[0]
    
    tempAwayG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'G_FPAPG'] += tempAwayG
    team_names_FPAPG.at[index_of_home, 'G_FPAPG'] += tempHomeG
    
    tempAwayF = opp_forward.loc[opp_forward['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeF = opp_forward.loc[opp_forward['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'F_FPAPG'] += tempAwayF
    team_names_FPAPG.at[index_of_home, 'F_FPAPG'] += tempHomeF
    
    tempAwayD = opp_defense.loc[opp_defense['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeD = opp_defense.loc[opp_defense['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'D_FPAPG'] += tempAwayD
    team_names_FPAPG.at[index_of_home, 'D_FPAPG'] += tempHomeD
    
# Gmean = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Fmean = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Dmean = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Gstd = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Fstd = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Dstd = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).std()

# for index, row in team_names_FPAPG.iterrows():
#     Gmodifier = (row['G_FPAPG']/row['gamesRemaining']) - Gmean
#     adjModifier = (Gmodifier - Gmean) / Gstd
    
#     team_names_FPAPG.at[index, 'Gsos'] = adjModifier
    
#     Fmodifier = (row['F_FPAPG']/row['gamesRemaining']) - Fmean
#     adjModifier = (Fmodifier - Fmean) / Fstd
    
#     team_names_FPAPG.at[index, 'Fsos'] = adjModifier
    
#     Dmodifier = (row['D_FPAPG']/row['gamesRemaining']) - Dmean
#     adjModifier = (Dmodifier - Dmean) / Dstd
    
#     team_names_FPAPG.at[index, 'Dsos'] = adjModifier
    
team_names_FPAPG['G_FPAPG_PG'] = team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['F_FPAPG_PG'] = team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['D_FPAPG_PG'] = team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['Gmod'] = team_names_FPAPG['G_FPAPG_PG'] / team_names_FPAPG['G_FPAPG_PG'].mean()
team_names_FPAPG['Fmod'] = team_names_FPAPG['F_FPAPG_PG'] / team_names_FPAPG['F_FPAPG_PG'].mean()
team_names_FPAPG['Dmod'] = team_names_FPAPG['D_FPAPG_PG'] / team_names_FPAPG['D_FPAPG_PG'].mean()
# team_names_FPAPG['GmodNorm'] = (team_names_FPAPG['Gmod'] - team_names_FPAPG['Gmod'].min()) / (team_names_FPAPG['Gmod'].max() - team_names_FPAPG['Gmod'].min())

team_names_FPAPG

,abbreviation,name,gamesPlayed,gamesRemaining,goalsFor,goalsAgainst,logo,conference,division,G_FPAPG,F_FPAPG,D_FPAPG,G_FPAPG_PG,F_FPAPG_PG,D_FPAPG_PG,Gmod,Fmod,Dmod
0,VGK,Vegas Golden Knights,31,51,107,78,https://assets.nhle.com/logos/nhl/svg/VGK_ligh...,Western,Pacific,88.12,803.84,461.47,1.727843,15.761569,9.048431,0.906364,0.988418,1.005465
1,NYR,New York Rangers,29,53,96,80,https://assets.nhle.com/logos/nhl/svg/NYR_ligh...,Eastern,Metropolitan,109.14,865.12,474.99,2.059245,16.323019,8.962075,1.080206,1.023626,0.995869
2,BOS,Boston Bruins,29,53,94,73,https://assets.nhle.com/logos/nhl/svg/BOS_ligh...,Eastern,Atlantic,101.57,852.20,476.71,1.916415,16.079245,8.994528,1.005282,1.008339,0.999475
3,VAN,Vancouver Canucks,31,51,116,76,https://assets.nhle.com/logos/nhl/svg/VAN_ligh...,Western,Pacific,95.13,787.44,447.62,1.865294,15.440000,8.776863,0.978466,0.968252,0.975288
4,LAK,Los Angeles Kings,27,55,97,67,https://assets.nhle.com/logos/nhl/svg/LAK_ligh...,Western,Pacific,109.77,893.20,510.82,1.995818,16.240000,9.287636,1.046934,1.018420,1.032045
5,TOR,Toronto Maple Leafs,28,54,106,90,https://assets.nhle.com/logos/nhl/svg/TOR_ligh...,Eastern,Atlantic,105.53,869.37,482.47,1.954259,16.099444,8.934630,1.025134,1.009606,0.992819
6,WPG,Winnipeg Jets,29,53,97,76,https://assets.nhle.com/logos/nhl/svg/WPG_ligh...,Western,Central,106.29,853.30,483.82,2.005472,16.100000,9.128679,1.051998,1.009641,1.014382
7,DAL,Dallas Stars,29,53,103,92,https://assets.nhle.com/logos/nhl/svg/DAL_ligh...,Western,Central,106.81,854.90,482.88,2.015283,16.130189,9.110943,1.057145,1.011534,1.012411
8,COL,Colorado Avalanche,30,52,107,94,https://assets.nhle.com/logos/nhl/svg/COL_ligh...,Western,Central,90.40,817.35,461.57,1.738462,15.718269,8.876346,0.911934,0.985702,0.986343
9,FLA,Florida Panthers,30,52,90,78,https://assets.nhle.com/logos/nhl/svg/FLA_ligh...,Eastern,Atlantic,94.21,833.40,462.35,1.811731,16.026923,8.891346,0.950368,1.005058,0.988009


# Adjusted remain based on sked

In [17]:
# for index, row in player_bios.iterrows():
#     if row['position'] == 'G':
#         player_log = allG_df.loc[allG_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics_g(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPPG'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Gmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
#     elif row['position'] == 'D':
#         player_log = all_df.loc[all_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPP60'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Dmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
#     else:
#         player_log = all_df.loc[all_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPP60'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Fmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']

# player_bios.sort_values('adjFantasyRemain')

for index, row in player_bios.iterrows():
    sosG = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Gmod'].iloc[0]
    sosF = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Fmod'].iloc[0]
    sosD = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Dmod'].iloc[0]
    if row['position'] == 'D':
        player_bios.at[index, 'adjSkedFantasyRemain'] = row['adjFantasyRemain'] * sosD
    elif row['position'] == 'G':
        player_bios.at[index, 'adjSkedFantasyRemain'] = row['adjFantasyRemain'] * sosG
    else:
        player_bios.at[index, 'adjSkedFantasyRemain'] = row['adjFantasyRemain'] * sosF
        
player_bios['adjFantasyRemain'] = round(player_bios['adjFantasyRemain'], 2)
player_bios['adjSkedFantasyRemain'] = round(player_bios['adjSkedFantasyRemain'], 2)

player_bios.sort_values('adjSkedFantasyRemain', ascending=False, inplace=True)
player_bios

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,last10FP,adjFantasyRemain,adjSkedFantasyRemain
0,8479318,A. Matthews,TOR,C,35008,27,95.9,9.86,3.55,0,...,L,75,215,99.907626,C,4024123,3.83,38.3,196.74,198.63
3,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,...,L,73,194,99.883457,C,3895074,4.14,41.4,185.40,184.77
1,8481559,J. Hughes,NJD,C,28599,23,76.1,9.58,3.31,0,...,L,71,175,99.775460,C,4565222,3.47,34.7,181.62,178.87
17,8476945,C. Hellebuyck,WPG,G,79316,22,75.4,3.42,3.43,0,...,L,76,207,99.454882,G,3020225,5.77,57.6,168.13,176.88
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,...,L,71,182,99.877782,RW,2563060,3.10,31.0,165.67,165.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,8475831,P. Grubauer,SEA,G,55322,17,-2.6,-0.17,-0.15,2,...,L,73,188,5.093937,G,5657,-0.88,-7.4,-8.95,-9.54
854,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,L,75,200,2.226018,G,5473,-1.93,-8.6,-15.44,-15.86
853,8477293,A. Raanta,CAR,G,44928,14,-9.2,-0.74,-0.66,0,...,L,72,195,5.982979,G,3037703,-1.43,-13.2,-20.37,-19.48
855,8477480,E. Comrie,BUF,G,20642,7,-13.2,-2.30,-1.89,0,...,L,73,183,0.240515,G,3042057,-2.30,-13.2,-20.70,-21.05


## Add new ranking --- MANUALLY FIX DATE!!!!

In [19]:
for index, row in rankings.iterrows():
    try:
        rankings.at[index,'fantasyRemain'] = player_bios.loc[player_bios['espnId'] == row['espnId']]['adjSkedFantasyRemain'].iloc[0]
    except:
        rankings.at[index,'fantasyRemain'] = 0


rankings['rank_2023-12-18'] = rankings['fantasyRemain'].rank(ascending=False, method='min').astype(int)
rankings 

rankings = rankings[column_order]
rankings.sort_values('rank_2023-12-18')

fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Benoit-Olivier Groulx': 'Bo Groulx',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
#     'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker',
    'Alex Barre-Boulet': 'Alex Barré-Boulet'
}

inverse_fixes = {value: key for key, value in fixes.items()}

rankings['FullName'].replace(fixes, inplace=True)
rankings.sort_values('rank_2023-12-18')

,espnId,FullName,fantasyRemain,rank_2023-12-18,rank_2023-12-11,rank_2023-12-04,rank_2023-11-27,rank_2023-11-20,rank_2023-11-13,rank_2023-11-06,rank_2023-10-30,rank_2023-10-23,rank_2023-09-01
0,4024123,Auston Matthews,198.63,1,4,7,2,3.0,1.0,1.0,2.0,2.0,5.0
2,3895074,Connor McDavid,184.77,2,2,4,4,2.0,3.0,2.0,1.0,1.0,1.0
4,4565222,Jack Hughes,178.87,3,11,5,6,4.0,5.0,5.0,5.0,7.0,13.0
71,3020225,Connor Hellebuyck,176.88,4,6,16,66,79.0,72.0,59.0,58.0,56.0,28.0
5,2563060,Nikita Kucherov,165.51,5,3,3,3,5.0,6.0,9.0,10.0,13.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,3042069,Spencer Martin,-5.32,1041,1040,681,805,771.0,661.0,651.0,1041.0,260.0,1041.0
635,5657,Philipp Grubauer,-9.54,1042,1042,693,787,752.0,625.0,699.0,603.0,260.0,603.0
1040,5473,Jack Campbell,-15.86,1043,1044,1045,820,807.0,1041.0,763.0,369.0,260.0,369.0
623,3037703,Antti Raanta,-19.48,1044,1038,690,789,614.0,611.0,680.0,521.0,260.0,521.0


In [20]:
file_name = f"data/rankings_worksheet_{(today).strftime('%Y-%m-%d')}.xlsx"
# file_name = f"data/manual-adjustments.xlsx"
rankings.to_excel(file_name, index=False)

# MANUALLY ADJUST DATE

In [21]:
# last_10['rank'] = last_10['fantasyPoints'].rank(method='first', ascending=False)
# last_10['rank'] = last_10['rank'].astype(int)
# last_10.head(40)
player_bios['last10rank'] = player_bios['last10FP'].rank(ascending=False).astype(int)
last_week = pd.read_excel('data/rankings-final-Dec13.xlsx')
last_week

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,adjSkedFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos
0,8478402,C. McDavid,EDM,C,29777,23,71.2,8.61,3.10,0,...,3895074,4.06,40.6,194.94,193.16,7,1,1,1,1
1,8476453,N. Kucherov,TBL,R,36222,28,97.7,9.71,3.49,0,...,2563060,3.65,36.5,187.80,186.16,12,2,4,2,1
2,8480069,C. Makar,COL,D,38098,26,82.2,7.77,3.16,0,...,4233563,2.91,29.1,166.14,165.91,21,3,2,3,1
3,8479318,A. Matthews,TOR,C,32236,25,82.7,9.24,3.31,0,...,4024123,3.03,30.3,183.35,183.85,17,4,5,4,2
4,8475660,C. Talbot,LAK,G,64183,18,80.0,4.49,4.44,0,...,5734,5.63,56.0,194.80,207.47,1,5,7,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,8475831,P. Grubauer,SEA,G,55322,17,-2.6,-0.17,-0.15,3,...,5657,-0.88,-7.4,-10.57,-11.14,844,846,811,846,71
846,8478872,K. Vejmelka,ARI,G,40134,12,-3.8,-0.34,-0.32,0,...,3942065,-1.43,-13.0,-15.47,-15.76,848,847,824,847,72
847,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,5473,-1.93,-8.6,-19.30,-20.11,845,848,829,848,73
848,8477480,E. Comrie,BUF,G,20642,7,-13.2,-2.30,-1.89,0,...,3042057,-2.30,-13.2,-23.00,-22.65,849,849,828,849,74


In [22]:
player_bios
remain_rank = 0
for index, row in player_bios.iterrows():
    try:
        rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-12-18'].iloc[0]
        last_rank = last_week.loc[last_week['espnId'] == row['espnId']]['remain_rank'].iloc[0]
#         past_10 = last_10.loc[rankings['espnId'] == row['espnId']]['rank'].iloc[0]
    except:
        rank = 300
        last_rank = 300
#         past_10 = 800
        
    remain_rank += 1
        
    player_bios.at[index, 'remain_rank'] =  remain_rank
    player_bios.at[index, 'last_rank'] =  last_rank
    player_bios.at[index, 'rank'] =  rank
#     player_bios.at[index, 'past_10'] =  past_10
    
not_ranked = player_bios.loc[player_bios['rank'] == 300]
for index, row in not_ranked.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")
    if row['remain_rank'] > 300:
        break

print('-------------')        

poor_rank = player_bios.loc[(player_bios['rank'] >= 250) & (player_bios['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")


C. Ingram: 29.0
N. Seeler: 153.0
B. Faber: 200.0
M. Carcone: 205.0
J. Bernard-Docker: 210.0
L. Belpedio: 235.0
K. Korczak: 250.0
H. Thrun: 262.0
G. Lindström: 264.0
N. Okhotiuk: 265.0
D. Voronkov: 277.0
A. Vlasic: 283.0
D. Mermis: 286.0
J. Danforth: 310.0
-------------
J. Benn: 254.0
P. Kurashev: 255.0
J. Drysdale: 256.0
B. Schenn: 258.0
T. Thompson: 261.0
A. Lee: 266.0
M. Fehervary: 272.0
J. Peterka: 273.0
L. Ullmark: 274.0
T. Myers: 278.0
S. Walker: 281.0
A. Svechnikov: 285.0
S. Gostisbehere: 292.0
A. Ekblad: 294.0
W. Johnston: 296.0
A. Killorn: 298.0
J. McBain: 299.0
T. Teravainen: 308.0
C. Perfetti: 319.0
J. Neighbours: 331.0
S. Lafferty: 338.0
A. Lowry: 347.0
B. Burns: 360.0
J. Brodzinski: 385.0
A. Georgiev: 396.0
I. Shesterkin: 399.0
P. Kane: 405.0
S. Skinner: 444.0
J. Oettinger: 673.0
A. Vasilevskiy: 764.0


In [23]:
def convert_to_slug(name):
    # Convert to lowercase and replace spaces with hyphens
    slug = name.lower().replace(' ', '-')
    return slug

In [24]:
player_bios['rank_by_pos'] = player_bios.groupby('default_pos')['rank'].rank(ascending=True)
player_bios.head(50)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,adjSkedFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos
0,8479318,A. Matthews,TOR,C,35008,27,95.9,9.86,3.55,0,...,4024123,3.83,38.3,196.74,198.63,10,1.0,4.0,1.0,1.0
3,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,...,3895074,4.14,41.4,185.40,184.77,8,2.0,1.0,2.0,2.0
1,8481559,J. Hughes,NJD,C,28599,23,76.1,9.58,3.31,0,...,4565222,3.47,34.7,181.62,178.87,14,3.0,11.0,3.0,3.0
17,8476945,C. Hellebuyck,WPG,G,79316,22,75.4,3.42,3.43,0,...,3020225,5.77,57.6,168.13,176.88,1,4.0,10.0,4.0,1.0
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,...,2563060,3.10,31.0,165.67,165.51,24,5.0,2.0,5.0,1.0
6,8477492,N. MacKinnon,COL,C,40820,30,91.9,8.10,3.06,0,...,3041969,3.36,33.6,164.32,161.97,16,6.0,6.0,6.0,4.0
4,8475660,C. Talbot,LAK,G,71457,20,80.2,4.04,4.01,0,...,5734,3.51,35.0,154.53,161.79,13,7.0,5.0,7.0,2.0
10,8480865,N. Dobson,NYI,D,45996,30,89.3,6.99,2.98,0,...,4352732,3.25,32.5,159.64,160.10,18,8.0,12.0,8.0,1.0
11,8480803,E. Bouchard,EDM,D,37541,28,78.7,7.55,2.81,0,...,4352722,3.21,32.1,158.94,159.48,21,9.0,8.0,9.0,2.0
15,8475786,Z. Hyman,EDM,L,31514,27,72.4,8.27,2.68,0,...,5509,3.39,33.9,157.50,156.96,15,10.0,24.0,10.0,2.0


# THIS IS THE STOP AND FIX SPOT

In [26]:
# player_bios.to_excel('data/manual-adjustments.xlsx', index=False)

player_bios_adjusted = pd.read_excel('data/manual-adjustments.xlsx')

# # vasil = {'playerId': 8476883, 'name': 'A. Vasilevskiy', 'team': 'TBL', 'position': 'G', 'default_pos': 'G', 'espnId': 2976847, 'remain_rank': 55, 'fullName': 'Andrei Vasilevskiy'}
# # player_bios_adjusted = player_bios_adjusted.append(vasil, ignore_index=True)
# kane = {'playerId': 8474141, 'name': 'P. Kane', 'team': 'DET', 'position': 'R', 'default_pos': 'RW', 'espnId': 3735, 'remain_rank': 125, 'fullName': 'Patrick Kane'}
# player_bios_adjusted = player_bios_adjusted.append(kane, ignore_index=True)
# for index, row in player_bios_adjusted.iterrows():
#     player_bios_adjusted.at[index, 'fullName'] = player_bios.loc[player_bios['playerId'] == row['playerId'], 'fullName'].iloc[0]
#     player_bios_adjusted.at[index, 'default_pos'] = player_bios.loc[player_bios['playerId'] == row['playerId'], 'default_pos'].iloc[0]
    
player_bios_adjusted['espnId'] = player_bios_adjusted['espnId'].fillna(0).astype(int)
player_bios_adjusted = player_bios_adjusted.sort_values('remain_rank', ascending=True)
player_bios_adjusted['rank'] = player_bios_adjusted['remain_rank'].rank(ascending=True, method='first')
player_bios_adjusted['rank_by_pos'] = player_bios_adjusted.groupby('default_pos')['rank'].rank(ascending=True)
player_bios_adjusted.head(60)

player_bios_adjusted

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,adjSkedFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos
0,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,...,3895074,4.14,41.4,185.40,184.77,8,1,1,1.0,1.0
1,8479318,A. Matthews,TOR,C,35008,27,95.9,9.86,3.55,0,...,4024123,3.83,38.3,196.74,198.63,10,2,4,2.0,2.0
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,...,2563060,3.10,31.0,165.67,165.51,24,3,2,3.0,1.0
3,8477956,D. Pastrnak,BOS,R,34133,29,86.9,9.17,3.00,0,...,3114778,2.58,25.8,151.58,152.84,46,4,7,4.0,2.0
4,8480069,C. Makar,COL,D,39139,27,83.9,7.72,3.11,0,...,4233563,2.62,26.2,153.23,151.13,43,5,3,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,8475831,P. Grubauer,SEA,G,55322,17,-2.6,-0.17,-0.15,2,...,5657,-0.88,-7.4,-8.95,-9.54,850,853,846,853.0,74.0
853,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,5473,-1.93,-8.6,-15.44,-15.86,851,854,848,854.0,75.0
854,8477293,A. Raanta,CAR,G,44928,14,-9.2,-0.74,-0.66,0,...,3037703,-1.43,-13.2,-20.37,-19.48,855,855,842,855.0,76.0
855,8477480,E. Comrie,BUF,G,20642,7,-13.2,-2.30,-1.89,0,...,3042057,-2.30,-13.2,-20.70,-21.05,854,856,849,856.0,77.0


In [27]:
team_names_only = team_names[['abbreviation', 'name']]

espn_teams = {
    'BOS': 'Bos',
    'VGK': 'Vgk',
    'NYR': 'NYR',
    'VAN': 'Van',
    'LAK': 'LA',
    'DAL': 'Dal',
    'FLA': 'Fla',
    'COL': 'Col',
    'WPG': 'Wpg',
    'TOR': 'Tor',
    'WSH': 'Wsh',
    'CAR': 'Car',
    'TBL': 'TB',
    'PHI': 'Phi',
    'DET': 'Det',
    'ANA': 'Ana',
    'ARI': 'Ari',
    'SEA': 'Sea',
    'NJD': 'NJ',
    'STL': 'StL',
    'NYI': 'NYI',
    'OTT': 'Ott',
    'PIT': 'Pit',
    'MTL': 'Mon',
    'BUF': 'Buf',
    'CGY': 'Cgy',
    'MIN': 'Min',
    'NSH': 'Nsh',
    'CBJ': 'Cls',
    'EDM': 'Edm',
    'CHI': 'Chi',
    'SJS': 'SJ'
}

team_names_only['espn_name'] = team_names_only['abbreviation'].map(espn_teams)
team_names_only

,abbreviation,name,espn_name
0,VGK,Vegas Golden Knights,Vgk
1,NYR,New York Rangers,NYR
2,BOS,Boston Bruins,Bos
3,VAN,Vancouver Canucks,Van
4,LAK,Los Angeles Kings,LA
5,TOR,Toronto Maple Leafs,Tor
6,WPG,Winnipeg Jets,Wpg
7,DAL,Dallas Stars,Dal
8,COL,Colorado Avalanche,Col
9,FLA,Florida Panthers,Fla


In [28]:
fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Benoit-Olivier Groulx': 'Bo Groulx',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
#     'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker',
    'Alex Barre-Boulet': 'Alex Barré-Boulet'
}

inverse_fixes = {value: key for key, value in fixes.items()}

player_bios_adjusted['fullName'].replace(inverse_fixes, inplace=True)

player_bios_adjusted

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,adjSkedFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos
0,8478402,C. McDavid,EDM,C,33567,26,80.0,8.58,3.08,0,...,3895074,4.14,41.4,185.40,184.77,8,1,1,1.0,1.0
1,8479318,A. Matthews,TOR,C,35008,27,95.9,9.86,3.55,0,...,4024123,3.83,38.3,196.74,198.63,10,2,4,2.0,2.0
2,8476453,N. Kucherov,TBL,R,40470,31,106.1,9.44,3.42,0,...,2563060,3.10,31.0,165.67,165.51,24,3,2,3.0,1.0
3,8477956,D. Pastrnak,BOS,R,34133,29,86.9,9.17,3.00,0,...,3114778,2.58,25.8,151.58,152.84,46,4,7,4.0,2.0
4,8480069,C. Makar,COL,D,39139,27,83.9,7.72,3.11,0,...,4233563,2.62,26.2,153.23,151.13,43,5,3,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,8475831,P. Grubauer,SEA,G,55322,17,-2.6,-0.17,-0.15,2,...,5657,-0.88,-7.4,-8.95,-9.54,850,853,846,853.0,74.0
853,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,5473,-1.93,-8.6,-15.44,-15.86,851,854,848,854.0,75.0
854,8477293,A. Raanta,CAR,G,44928,14,-9.2,-0.74,-0.66,0,...,3037703,-1.43,-13.2,-20.37,-19.48,855,855,842,855.0,76.0
855,8477480,E. Comrie,BUF,G,20642,7,-13.2,-2.30,-1.89,0,...,3042057,-2.30,-13.2,-20.70,-21.05,854,856,849,856.0,77.0


In [29]:
player_bios_adjusted
poor_rank = player_bios_adjusted.loc[(player_bios_adjusted['rank'] >= 250) & (player_bios_adjusted['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['last_rank']}")

S. Skinner: 235
J. Dickinson: 168
J. Neighbours: 162
J. McBain: 245
P. Kurashev: 225
M. Jankowski: 242
J. Drysdale: 224
B. Schenn: 218
A. Lee: 182
M. Fehervary: 237
J. Peterka: 220
T. Myers: 239
S. Walker: 232
S. Gostisbehere: 137
A. Ekblad: 180
A. Killorn: 240
T. Teravainen: 238
C. Perfetti: 249
S. Lafferty: 241
A. Lowry: 230
J. Brodzinski: 188


In [30]:
# 1. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C1) <br />

ranking_module = pd.DataFrame()

for index, row in player_bios_adjusted.iterrows():
    base_string = '<a href="http://www.espn.com/nhl/player/_/id/'
    rank = str(int(row['rank']))
    Pid = str(row['espnId']) + '/'
    name = row['fullName']
    slug = convert_to_slug(name)
    pos = row['default_pos']
    team = team_names_only.loc[team_names_only['abbreviation'] == row['team']]['espn_name'].iloc[0]
    pos_rank = pos + str(int(row['rank_by_pos']))
    
    assemble = rank + '. ' + base_string + Pid + slug + '">' + name + '</a>, ' + pos + ', ' + team + ' (' + pos_rank + ')'
    
    temp = pd.DataFrame({'column': [assemble]})
    
    ranking_module = pd.concat([ranking_module, temp])
    
    print(f"{assemble}")
    
    if int(rank) == 250:
        break


1. <a href="http://www.espn.com/nhl/player/_/id/3895074/connor-mcdavid">Connor McDavid</a>, C, Edm (C1)
2. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C2)
3. <a href="http://www.espn.com/nhl/player/_/id/2563060/nikita-kucherov">Nikita Kucherov</a>, RW, TB (RW1)
4. <a href="http://www.espn.com/nhl/player/_/id/3114778/david-pastrnak">David Pastrnak</a>, RW, Bos (RW2)
5. <a href="http://www.espn.com/nhl/player/_/id/4233563/cale-makar">Cale Makar</a>, D, Col (D1)
6. <a href="http://www.espn.com/nhl/player/_/id/3041969/nathan-mackinnon">Nathan MacKinnon</a>, C, Col (C3)
7. <a href="http://www.espn.com/nhl/player/_/id/4565222/jack-hughes">Jack Hughes</a>, C, NJ (C4)
8. <a href="http://www.espn.com/nhl/player/_/id/3020225/connor-hellebuyck">Connor Hellebuyck</a>, G, Wpg (G1)
9. <a href="http://www.espn.com/nhl/player/_/id/5734/cam-talbot">Cam Talbot</a>, G, LA (G2)
10. <a href="http://www.espn.com/nhl/player/_/id/4352732/noah-dobson">Noah

# MANUALLY ADJUST ITERATION

In [34]:
fileName = f"data/2024Rankings-09-{(today+timedelta(days=1)).strftime('%b%d')}.xlsx"
ranking_module.to_excel(fileName, index=False, header=None)

In [32]:
for i in range(250):
    print(f"{player_bios_adjusted.iloc[i, 1]}: {int(player_bios_adjusted.iloc[i, 26]) - int(player_bios_adjusted.iloc[i, 25])}")

C. McDavid: 0
A. Matthews: 2
N. Kucherov: -1
D. Pastrnak: 3
C. Makar: -2
N. MacKinnon: 0
J. Hughes: 4
C. Hellebuyck: 2
C. Talbot: -4
N. Dobson: 2
E. Bouchard: -3
A. Panarin: 2
W. Nylander: 3
J. Miller: -5
Z. Hyman: 9
M. Marner: 19
S. Reinhart: -4
L. Draisaitl: -3
J. Eichel: 3
R. Josi: 0
B. Tkachuk: -4
A. Fox: -4
C. McAvoy: 5
R. Dahlin: 1
B. Boeser: 14
M. Seider: -7
M. Rantanen: -1
E. Pettersson: -5
A. Barkov: 11
T. Demko: 230
M. Rielly: 7
J. Trouba: -11
F. Forsberg: -4
J. Morrissey: -7
M. Zibanejad: 13
Q. Hughes: -6
K. Kaprizov: -6
J. Pavelski: -5
S. Crosby: 7
J. Saros: 32
A. Pietrangelo: 1
E. Kane: -1
J. Guentzel: 25
A. Hill: -1
M. Weegar: 20
J. Walman: 14
I. Sorokin: -11
J. Oettinger: -11
J. Eriksson Ek: -15
I. Shesterkin: -6
M. Scheifele: 7
J. Bratt: 10
A. Kopitar: 1
V. Trocheck: 10
B. Horvat: 31
D. Nurse: 10
T. Stützle: -10
J. Chychrun: 9
M. Tkachuk: 2
J. Tavares: 10
R. Andersson: 12
S. Stamkos: 31
M. Barzal: 17
J. Robertson: -9
T. Jarry: -15
B. Marchand: -9
J. Swayman: -4
A. Vasil

In [33]:
fileName = f"data/rankings-final-{(today+timedelta(days=1)).strftime('%b%d')}.xlsx"
player_bios_adjusted.to_excel(fileName, index=False)